In [1]:
from pathlib import Path

import torch
import pefile
import pandas as pd
from tqdm import tqdm

In [2]:
NOTEBOOKS_DIR = Path().resolve()
PROJECT_DIR = NOTEBOOKS_DIR.parent
DATA_DIR = PROJECT_DIR / 'data'
DATASET_DIR = DATA_DIR / 'pe-machine-learning-dataset'

SAMPLES_WITH_REPORTS_DIR = DATASET_DIR / 'samples_with_reports'
SAMPLES_PICTURES = DATASET_DIR / 'samples_pictures'
SAMPLES_PICTURES.mkdir(exist_ok=True)

RANDOM_STATE = 741

In [3]:
df = pd.read_parquet(DATA_DIR / 'labeled_df.parquet')

In [4]:
for filename in tqdm(df.filename, total=df.shape[0]):
    filename = str(filename)
    with open(SAMPLES_WITH_REPORTS_DIR / filename, 'rb') as _file:
        data_ch1 = list(_file.read(224 * 224))
        data_ch2 = data_ch1.copy()
        data_ch3 = data_ch1.copy()

    try:
        pe = pefile.PE(SAMPLES_WITH_REPORTS_DIR / filename)

        for section in pe.sections:
            if section.Name.startswith(b'.text'):
                data_ch2 = list(pe.__data__[section.get_PointerToRawData_adj():224*224])
                break
        
        for section in pe.sections:
            if section.Name.startswith(b'.data'):
                data_ch3 = list(pe.__data__[section.get_PointerToRawData_adj():224*224])
                break
    except:
        pass

    for data in (data_ch1, data_ch2, data_ch3):
        padding = [0] * (224*224 - len(data))
        data.extend(padding)

    img = torch.stack(
        [
            torch.reshape(input=torch.tensor(data_ch1, dtype=torch.uint8), shape=(1, 224, 224)),
            torch.reshape(input=torch.tensor(data_ch2, dtype=torch.uint8), shape=(1, 224, 224)),
            torch.reshape(input=torch.tensor(data_ch3, dtype=torch.uint8), shape=(1, 224, 224)),
        ],
        dim=1
    )

    torch.save(img, SAMPLES_PICTURES / filename)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 39090/39090 [1:25:48<00:00,  7.59it/s]
